In [1]:
from pyspark.sql import SparkSession
import os
# spark = SparkSession.builder \
#     .appName("TCPH-Notebook") \
#     .master("spark://spark-master:7077") \
#     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
#     .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
#     .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
#     .config("spark.hadoop.fs.s3a.path.style.access", "true") \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .getOrCreate()
spark = SparkSession.builder \
        .appName("TCPH-Notebook") \
        .master("spark://spark-master:7077") \
        .config("spark.cores.max", "2") \
        .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
        .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
        .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .getOrCreate()

In [2]:
spark

In [5]:
df_read = spark.read.parquet("s3a://bronze/customer")
df_read.show(5)

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
|c_custkey|            c_name|           c_address|c_nationkey|        c_phone|c_acctbal|c_mktsegment|           c_comment|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
|        1|Customer#000000001|   IVhzIApeRb ot,c,E|         15|25-989-741-2988|   711.56|    BUILDING|to the even, regu...|
|        2|Customer#000000002|XSTf4,NCwDVaWNe6t...|         13|23-768-687-3665|   121.65|  AUTOMOBILE|l accounts. blith...|
|        3|Customer#000000003|        MG9kdTD2WBHm|          1|11-719-748-3364|  7498.12|  AUTOMOBILE| deposits eat sly...|
|        4|Customer#000000004|         XxVSJsLAGtn|          4|14-128-190-5944|  2866.83|   MACHINERY| requests. final,...|
|        5|Customer#000000005|KvpyuHCplrB84WgAi...|          3|13-750-942-6364|   794.47|   HOUSEHOLD|n accounts will h...|
+-------

In [3]:
spark.version

'3.5.0'

In [6]:
# List all tables
tables = ['nation', 'region', 'customer', 'orders', 'lineitem', 'part', 'partsupp', 'supplier']

# Check size và row count
for table in tables:
    df = spark.read.parquet(f"s3a://bronze/{table}")
    print(f"{table}: {df.count():,} rows, {df.rdd.getNumPartitions()} partitions")

nation: 25 rows, 1 partitions
region: 5 rows, 1 partitions
customer: 150,000 rows, 2 partitions
orders: 1,500,000 rows, 2 partitions
lineitem: 2,000,000 rows, 2 partitions
part: 200,000 rows, 2 partitions
partsupp: 800,000 rows, 2 partitions
supplier: 10,000 rows, 1 partitions


In [8]:
for table in tables:
    df = spark.read.parquet(f"s3a://bronze/{table}")
    # Schema validation
    df.printSchema()
    df.dtypes

root
 |-- n_nationkey: long (nullable = true)
 |-- n_name: string (nullable = true)
 |-- n_regionkey: long (nullable = true)
 |-- n_comment: string (nullable = true)

root
 |-- r_regionkey: long (nullable = true)
 |-- r_name: string (nullable = true)
 |-- r_comment: string (nullable = true)

root
 |-- c_custkey: long (nullable = true)
 |-- c_name: string (nullable = true)
 |-- c_address: string (nullable = true)
 |-- c_nationkey: long (nullable = true)
 |-- c_phone: string (nullable = true)
 |-- c_acctbal: double (nullable = true)
 |-- c_mktsegment: string (nullable = true)
 |-- c_comment: string (nullable = true)

root
 |-- o_orderkey: long (nullable = true)
 |-- o_custkey: long (nullable = true)
 |-- o_orderstatus: string (nullable = true)
 |-- o_totalprice: double (nullable = true)
 |-- o_orderdate: date (nullable = true)
 |-- o_orderpriority: string (nullable = true)
 |-- o_clerk: string (nullable = true)
 |-- o_shippriority: long (nullable = true)
 |-- o_comment: string (nullable 

In [9]:
# Numeric columns statistics
df.describe().show()

# Cardinality check
for col in df.columns:
    distinct_count = df.select(col).distinct().count()
    print(f"{col}: {distinct_count} distinct values")

+-------+-----------------+------------------+--------------------+-----------------+---------------+-----------------+--------------------+
|summary|        s_suppkey|            s_name|           s_address|      s_nationkey|        s_phone|        s_acctbal|           s_comment|
+-------+-----------------+------------------+--------------------+-----------------+---------------+-----------------+--------------------+
|  count|            10000|             10000|               10000|            10000|          10000|            10000|               10000|
|   mean|           5000.5|              NULL|                NULL|          11.9353|           NULL|4510.354864999992|                NULL|
| stddev|2886.895679907165|              NULL|                NULL|7.240756643174559|           NULL|3168.079218448369|                NULL|
|    min|                1|Supplier#000000001|  9aW1wwnBJJPnCx,...|                0|10-102-116-6785|          -998.22| about the blithe...|
|    max|    

In [10]:
# Check nulls for all tables
from pyspark.sql.functions import col, sum as spark_sum

tables = ['nation', 'region', 'customer', 'orders', 'lineitem', 'part', 'partsupp', 'supplier']

for table in tables:
   df = spark.read.parquet(f"s3a://bronze/{table}")
   
   # Count nulls per column
   null_counts = []
   for column in df.columns:
       null_count = df.filter(col(column).isNull()).count()
       if null_count > 0:
           null_counts.append(f"{column}: {null_count}")
   
   # Print results
   print(f"\n{table.upper()} ({df.count():,} rows):")
   if null_counts:
       for nc in null_counts:
           print(f"  {nc}")
   else:
       print("  ✓ No nulls")


NATION (25 rows):
  ✓ No nulls

REGION (5 rows):
  ✓ No nulls

CUSTOMER (150,000 rows):
  ✓ No nulls

ORDERS (1,500,000 rows):
  ✓ No nulls

LINEITEM (2,500,000 rows):
  ✓ No nulls

PART (200,000 rows):
  ✓ No nulls

PARTSUPP (800,000 rows):
  ✓ No nulls

SUPPLIER (10,000 rows):
  ✓ No nulls
